## Serving Tensorflow

TensorFlow actually has a built-in method to serve models, which is already optimized for use in production. It's also packaged as something that can easily be deployed on top of OpenShift

In this notebook, we'll set it up, and use a client to access it

First, we can log into the OpenShift cluster we're running on

In [7]:
!oc login 'https://api.cluster-7bf4.7bf4.openshiftworkshop.com:6443' --username='opentlc-mgr' \
               --password='r3dh4t1!' --insecure-skip-tls-verify='true'
!oc project "summit-user-${JUPYTERHUB_USER}"

Login successful.

You have access to 60 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "summit-user-6607d53f-fab4-4017-9496-5f253e3393ad".
Already on project "summit-user-6607d53f-fab4-4017-9496-5f253e3393ad" on server "https://api.cluster-7bf4.7bf4.openshiftworkshop.com:6443".


Now we can create a new OpenShift app. TensorFlow contains support for serving a model stored in S3, so we can take advantage of that to serve the model from

In [8]:
!oc new-app --name="tf-serving" \
             --docker-image="tensorflow/serving" \
             -e MODEL_BASE_PATH="s3://${JUPYTERHUB_USER}" \
             -e MODEL_NAME="mnist-model" \
             -e AWS_LOG_LEVEL=3 \
             -e AWS_ACCESS_KEY_ID="${ACCESSKEY}" \
             -e AWS_SECRET_ACCESS_KEY="${SECRETKEY}" \
             -e AWS_REGION="us-east-1" \
             -e S3_ENDPOINT="$(echo ${RGW_API_ENDPOINT} | sed 's%http://%%')" \
             -e S3_USE_HTTPS=0 \
             -e S3_VERIFY_SSL=0

--> Found Docker image 38bee21 (2 months old) from Docker Hub for "tensorflow/serving"

    * An image stream tag will be created as "tf-serving:latest" that will track this image
    * This image will be deployed in deployment config "tf-serving"
    * Ports 8500/tcp, 8501/tcp will be load balanced by service "tf-serving"
      * Other containers can access this service through the hostname "tf-serving"
    * WARNING: Image "tensorflow/serving" runs as the 'root' user which may not be permitted by your cluster administrator

--> Creating resources ...
    deploymentconfig.apps.openshift.io "tf-serving" created
    service "tf-serving" created
--> Success
    Application is not exposed. You can expose services to the outside world by executing one or more of the commands below:
     'oc expose svc/tf-serving' 
    Run 'oc status' to view your app.


The port needs to be exposed, since it exposes multiple ports, we'll redirect 8501 (the JSON api) to 80 for ease of use. This will allow us to verify it works via the browser.

In [9]:
!oc expose svc/tf-serving --port 8501

route.route.openshift.io/tf-serving exposed


Using `oc status` will let us see the URL that is exposed. Navigating to the url in the browser should return a 404, so instead navigate to `/v1/models/mnist-model`. It should return some metadata about the service

In [10]:
!oc status | grep tf-serving

http://tf-serving-summit-user-6607d53f-fab4-4017-9496-5f253e3393ad.apps.cluster-7bf4.7bf4.openshiftworkshop.com to pod port 8501 (svc/tf-serving)
  dc/tf-serving deploys istag/tf-serving:latest 


We can also double check it by curling the internal url or hostname, for which we'll want to use the internal port 8501

In [12]:
!curl "$(oc get services | grep 'tf-serving' | awk '{print $3}'):8501/v1/models/mnist-model"

{
 "model_version_status": [
  {
   "version": "1557258102",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}


### Testing the model with a Flask App

To show some interactive testing, we have developed a small webapp that allows a user to draw a digit by hand and submit it to the model for classification.

The model takes in the image, greyscales it, pads it, normalizes it, etc, and transforms it into the format that the model expects, then POSTs it to the model's API

In [13]:
!oc new-app --name "mnist-flask" "quay.io/llasmith/summit-workshop-flask-mnist:latest"
!oc expose svc/mnist-flask

--> Found Docker image cecf794 (2 hours old) from quay.io for "quay.io/llasmith/summit-workshop-flask-mnist:latest"

    * An image stream tag will be created as "mnist-flask:latest" that will track this image
    * This image will be deployed in deployment config "mnist-flask"
    * Port 5000/tcp will be load balanced by service "mnist-flask"
      * Other containers can access this service through the hostname "mnist-flask"
    * WARNING: Image "quay.io/llasmith/summit-workshop-flask-mnist:latest" runs as the 'root' user which may not be permitted by your cluster administrator

--> Creating resources ...
    deploymentconfig.apps.openshift.io "mnist-flask" created
    service "mnist-flask" created
--> Success
    Application is not exposed. You can expose services to the outside world by executing one or more of the commands below:
     'oc expose svc/mnist-flask' 
    Run 'oc status' to view your app.
route.route.openshift.io/mnist-flask exposed


To really experience the model, navigate to the URL that we exposed in the browser

Note: The app may take a few moments to spin up

In [14]:
!oc status | grep mnist

http://mnist-flask-summit-user-6607d53f-fab4-4017-9496-5f253e3393ad.apps.cluster-7bf4.7bf4.openshiftworkshop.com to pod port 5000-tcp (svc/mnist-flask)
  dc/mnist-flask deploys istag/mnist-flask:latest 
